# Quantum Computing

In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
# Imports
from qiskit import IBMQ, Aer, assemble
from qiskit.tools.monitor import job_monitor
from qiskit.providers.ibmq import least_busy
import qiskit as q
# from qiskit.tools.visualization import plot_bloch_sphere
from qiskit.visualization import plot_histogram, matplotlib
import matplotlib.pyplot as plt
from utils import dotdict
from DeutschJozsa.DeutschJozsa import DeutschJozsa
from BernsteinVazirani.BernsteinVazirani import BernsteinVazirani
from Simons.Simons import Simons
import os

In [ ]:
# Config
args = dotdict()
args.location = "local"  # local or remote
args.local_backend = "qasm_simulator"
args.remote_backend = ""
args.shots = 500

In [ ]:
base_dir = 'imgs/'
circuit_dir = base_dir + 'alg/'

if not os.path.exists(base_dir):
    os.mkdir(base_dir)

if not os.path.exists(circuit_dir):
    os.mkdir(circuit_dir)

In [ ]:
def save_circuit(circuit, name=None):
    if name is None:
        name = alg.name + '_alg'
    return circuit.draw(output="mpl").savefig(circuit_dir + name + '.pdf')

## Entanglement

In [ ]:
qc = q.QuantumCircuit(2)
qc.h(0)
qc.cx(0, 1)
print(qc)

In [ ]:
save_circuit(qc, "entanglement")

In [ ]:
# Run on simulation
qobj = assemble(qc)
svsim = Aer.get_backend('statevector_simulator')
result = svsim.run(qobj).result()
plot_histogram(result.get_counts())

In [ ]:
# Run on remote QC
IBMQ.save_account(open("token.txt", "r").read())
factory = IBMQ.load_account()
devices = factory.backends(filters=lambda x: x.configuration().n_qubits > 1 and
                                                   not x.configuration().simulator)
backend = least_busy(devices)
job = q.execute(qc, backend, shots=50)
job_monitor(job)
result = job.result()
plot_histogram(result.get_counts())

In [ ]:
print(job.result().()

## DeutschJozsa

In [ ]:
# Config
args.deutsch_jozsa_n = 3
args.deutsch_jozsa_oracle = 'constant' # balanced or constant

In [ ]:
# Algorithm
alg = DeutschJozsa(args)

In [ ]:
# Circuit
save_circuit(alg.circuit)

In [ ]:
# Run
result = alg.run()

In [ ]:
# Result
counts = result.get_counts(alg.circuit)
plot_histogram([counts])

In [ ]:
# Save results.
alg.save_results()

## Bernstein-Vazirani Algorithm

In [ ]:
# Config
args.bernstein_vazirani_s = '1001'

In [ ]:
# Algorithm
alg = BernsteinVazirani(args)

In [ ]:
# Circuit
save_circuit(alg.circuit)

In [ ]:
# Run
result = alg.run()

In [ ]:
# Result
counts = result.get_counts(alg.circuit)
plot_histogram([counts])

In [ ]:
# Save results.
alg.save_results()

## Simon's Algorithm

In [ ]:
# Config
args.simons_b = '1001'

In [ ]:
# Algorithm
alg = Simons(args)

In [ ]:
# Circuit
save_circuit(alg.circuit)

In [ ]:
# Run
result = alg.run()

In [ ]:
# Result
counts = result.get_counts(alg.circuit)
plot_histogram([counts])

In [ ]:
# Save results.
alg.save_results()

## Fourier?